In [ ]:
# Calvano et al. (2020) Q-Learning Replication (Colab)

このノートブックは Calvano et al. (2020) の Q学習による価格付け実験を Google Colab 上で再現します。

**論文**: Calvano, E., Calzolari, G., Denicolò, V., & Pastorello, S. (2020). Artificial Intelligence, Algorithmic Pricing, and Collusion. *American Economic Review*, 110(10), 3267-3297.

---

## 📋 実行手順

1. **セットアップ**: セクション1-2を順番に実行
2. **ベンチマーク計算**: セクション3で理論値を確認
3. **実験選択**: セクション4で実験レベルを選択
   - **クイックテスト**: 100エピソード（約5分）
   - **フル実験**: 50,000エピソード（8-12時間）
4. **結果確認**: セクション5で学習結果を可視化
5. **並列実験**: セクション6でTable A.2を再現（オプション）


In [ ]:
## 1. セットアップ（リポジトリのクローン＆依存パッケージのインストール）


In [ ]:
# リポジトリをクローン（並列実行対応ブランチ）
!git clone -b parallel-colab https://github.com/Yusei406/calvano-thesis.git
%cd calvano-thesis

# 現在のディレクトリを確認
!pwd
!ls -la

# 依存パッケージをインストール
!pip install -r requirements.txt

# パッケージが正しくインストールされたか確認
import sys
import os
print(f"Python path: {sys.path}")

# 現在の作業ディレクトリを取得
current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")

# myprojectディレクトリの存在確認
if os.path.exists('myproject'):
    print("✅ myprojectディレクトリが見つかりました")
    print("📁 myproject内容:")
    !ls -la myproject/
else:
    print("❌ myprojectディレクトリが見つかりません")
    print("📁 現在のディレクトリ内容:")
    !ls -la

print("✅ セットアップ完了")


In [ ]:
## 2. モジュールのインポート


In [ ]:
import sys
import os

# 現在のディレクトリをPythonパスに追加
sys.path.append('.')
sys.path.append(os.getcwd())

print(f"📍 現在のディレクトリ: {os.getcwd()}")
print(f"📁 myprojectの存在確認: {os.path.exists('myproject')}")

# 必要なモジュールをインポート
try:
    from myproject.env import DemandEnvironment
    from myproject.agent import QLearningAgent
    from myproject.train import train_agents
    print("✅ myprojectモジュールのインポート成功")
except ImportError as e:
    print(f"❌ myprojectモジュールのインポートエラー: {e}")
    print("📋 利用可能なファイル:")
    !find . -name "*.py" | head -10
    raise

# 標準ライブラリとサードパーティライブラリ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

print("✅ 全モジュールのインポート完了")


In [ ]:
## 3. 環境セットアップとベンチマーク計算（Nash/協調均衡）


In [ ]:
# Calvanoパラメータで環境を作成
env = DemandEnvironment(
    demand_intercept=0.0,   # a_0 (outside option)
    product_quality=2.0,    # a_i (product quality)
    demand_slope=0.25,      # μ (price sensitivity)
    marginal_cost=1.0       # c (marginal cost)
)

# Nash均衡・協調均衡を計算
nash_eq = env.get_nash_equilibrium()
coop_eq = env.get_collusive_outcome()

print('📊 ベンチマーク均衡の計算結果:')
print()
print('Nash Equilibrium:')
print(f'  Price: {nash_eq["prices"][0]:.3f}')
print(f'  Individual Profit: {nash_eq["individual_profit"]:.3f}')
print(f'  Joint Profit: {nash_eq["joint_profit"]:.3f}')
print()
print('Cooperative Equilibrium:')
print(f'  Price: {coop_eq["prices"][0]:.3f}')
print(f'  Individual Profit: {coop_eq["individual_profit"]:.3f}')
print(f'  Joint Profit: {coop_eq["joint_profit"]:.3f}')


In [ ]:
## 4. Q学習エージェントの学習実験

⚠️ **重要**: 下記のコメントアウトを調整して実験レベルを選択してください

- **クイックテスト**: 100エピソード（約5分）- デフォルトで有効
- **フル実験**: 50,000エピソード（8-12時間）- コメントアウトを外して使用


In [ ]:
# 1. クイックテスト（約5分）
print('🚀 クイックテスト実験を開始...')
agents, env, history = train_agents(
    n_episodes=100,
    iterations_per_episode=1000,
    learning_rate=0.15,
    discount_factor=0.95,
    epsilon_decay_beta=4e-6,
    memory_length=1,
    verbose=True
)

# 2. フル実験（8-12時間）- コメントアウトを外して使用
# print('🚀 フル実験（論文レベル）を開始...')
# print('⚠️  予想実行時間: 8-12時間')
# agents, env, history = train_agents(
#     n_episodes=50000,
#     iterations_per_episode=25000,
#     learning_rate=0.15,
#     discount_factor=0.95,
#     epsilon_decay_beta=4e-6,
#     memory_length=1,
#     verbose=True
# )

print('\n✅ 学習完了!')
print(f'Final individual profit: {history["training_summary"]["final_individual_profit"]:.4f}')
print(f'Final joint profit: {history["training_summary"]["final_joint_profit"]:.4f}')
print(f'Nash ratio (individual): {history["training_summary"]["nash_ratio_individual"]:.3f}')


In [ ]:
## 5. 学習履歴の可視化


In [ ]:
# 学習結果のプロット
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 個別利益の推移
axes[0, 0].plot(history['episodes'], history['individual_profits'], 'b-', linewidth=2)
axes[0, 0].axhline(y=nash_eq['individual_profit'], color='r', linestyle='--', label='Nash', linewidth=2)
axes[0, 0].axhline(y=coop_eq['individual_profit'], color='g', linestyle='--', label='Cooperative', linewidth=2)
axes[0, 0].set_title('Individual Profits', fontsize=14)
axes[0, 0].set_xlabel('Episode')
axes[0, 0].set_ylabel('Profit')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 合計利益の推移
axes[0, 1].plot(history['episodes'], history['joint_profits'], 'b-', linewidth=2)
axes[0, 1].axhline(y=nash_eq['joint_profit'], color='r', linestyle='--', label='Nash', linewidth=2)
axes[0, 1].axhline(y=coop_eq['joint_profit'], color='g', linestyle='--', label='Cooperative', linewidth=2)
axes[0, 1].set_title('Joint Profits', fontsize=14)
axes[0, 1].set_xlabel('Episode')
axes[0, 1].set_ylabel('Profit')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Nash比
nash_ratios = [p / nash_eq['individual_profit'] for p in history['individual_profits']]
axes[1, 0].plot(history['episodes'], nash_ratios, 'orange', linewidth=2)
axes[1, 0].axhline(y=1.0, color='r', linestyle='--', label='Nash', linewidth=2)
axes[1, 0].set_title('Individual Profit / Nash Profit', fontsize=14)
axes[1, 0].set_xlabel('Episode')
axes[1, 0].set_ylabel('Ratio')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Epsilon減衰
axes[1, 1].plot(history['episodes'], history['epsilon_values'], 'purple', linewidth=2)
axes[1, 1].set_title('Epsilon Decay', fontsize=14)
axes[1, 1].set_xlabel('Episode')
axes[1, 1].set_ylabel('Epsilon')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('📈 最終結果:')
print(f'Individual Profit: {history["training_summary"]["final_individual_profit"]:.4f}')
print(f'Joint Profit: {history["training_summary"]["final_joint_profit"]:.4f}')
print(f'Nash Ratio: {history["training_summary"]["nash_ratio_individual"]:.3f}')

# 協調均衡との比較を手動計算
coop_ratio_individual = history["training_summary"]["final_individual_profit"] / coop_eq['individual_profit']
coop_ratio_joint = history["training_summary"]["final_joint_profit"] / coop_eq['joint_profit']
print(f'Cooperative Ratio (Individual): {coop_ratio_individual:.3f}')
print(f'Cooperative Ratio (Joint): {coop_ratio_joint:.3f}')


In [ ]:
## 6. 並列実験でTable A.2を再現（オプション）

**論文の完全再現**を行う場合は、以下のフル実験を実行してください：

- **クイックテスト**: 1,000エピソード × 2シード（約10分）
- **フル実験**: 50,000エピソード × 10シード（8-12時間）


In [ ]:
# 並列実験スクリプトの存在確認
import os
script_path = 'myproject/scripts/table_a2_parallel.py'
if os.path.exists(script_path):
    print(f"✅ 並列実験スクリプトが見つかりました: {script_path}")
    
    # 1. クイックテスト並列実験（約10分）
    print("🚀 クイックテスト並列実験を開始...")
    !python -m myproject.scripts.table_a2_parallel --episodes 1000 --n-seeds 2 --n-sessions 2 --max-workers 2
    
    # 2. フル実験（8-12時間）- 論文の完全再現（コメントアウトを外して使用）
    # print("🚀 フル並列実験（Table A.2完全再現）を開始...")
    # print("📊 パラメータ: 50,000エピソード × 10シード")
    # print("⏰ 予想実行時間: 8-12時間")
    # !python -m myproject.scripts.table_a2_parallel --episodes 50000 --n-seeds 10 --n-sessions 4 --max-workers 4
    
    print("✅ 並列実験完了!")
else:
    print(f"❌ 並列実験スクリプトが見つかりません: {script_path}")
    print("📋 利用可能なスクリプト:")
    !find myproject -name "*.py" -type f


In [ ]:
## 7. 結果の確認とダウンロード


In [ ]:
# 結果ファイルの確認
import os
try:
    from google.colab import files
    colab_available = True
    print("✅ Google Colab環境を検出")
except ImportError:
    colab_available = False
    print("ℹ️  ローカル環境で実行中")

print("\n📁 生成されたファイル:")
if os.path.exists('results'):
    for root, dirs, files_list in os.walk('results'):
        for file in files_list:
            if file.endswith('.csv'):
                filepath = os.path.join(root, file)
                print(f"  {filepath}")
                
    # CSVファイルの内容を表示
    if os.path.exists('results/table_a2_parallel.csv'):
        print("\n📊 Table A.2 結果:")
        df = pd.read_csv('results/table_a2_parallel.csv')
        print(df)
        
        # 期待値との比較
        print("\n🎯 論文の期待値:")
        print("Individual profit: 0.18 ± 0.03")
        print("Joint profit: 0.26 ± 0.04")
        
        # ファイルダウンロード（Colab環境の場合のみ）
        if colab_available:
            print("\n⬇️  結果ファイルをダウンロード:")
            files.download('results/table_a2_parallel.csv')
        else:
            print(f"\n📄 結果ファイルの場所: {os.path.abspath('results/table_a2_parallel.csv')}")
    else:
        print("⚠️  results/table_a2_parallel.csv が見つかりません。")
        print("📋 resultsディレクトリの内容:")
        !ls -la results/
else:
    print("❌ resultsディレクトリが見つかりません。")
    print("📋 現在のディレクトリの内容:")
    !ls -la


In [ ]:
---

## 📚 参考情報

### 期待される結果（論文Table A.2）
- **Individual profit**: 0.18 ± 0.03
- **Joint profit**: 0.26 ± 0.04
- **Nash ratio**: > 100% (協調的行動を示唆)

### パラメータ設定
- **環境**: a₀=0.0, aᵢ=2.0, μ=0.25, c=1.0
- **Q学習**: α=0.15, γ=0.95, β=4×10⁻⁶, memory=1

### 実行時間の目安
- **クイックテスト**: 100エピソード（約5分）
- **中規模実験**: 1,000エピソード（約30分）
- **フル実験**: 50,000エピソード（8-12時間）

**🎓 論文**: Calvano, E., Calzolari, G., Denicolò, V., & Pastorello, S. (2020). *Artificial Intelligence, Algorithmic Pricing, and Collusion*. American Economic Review, 110(10), 3267-3297.


In [ ]:
## 🔧 トラブルシューティング

### よくあるエラーと解決方法

**1. `ModuleNotFoundError: No module named 'myproject'`**
- セクション1のセットアップセルを正しく実行したか確認
- `%cd calvano-thesis` が正しく動作しているか確認
- 以下のコードで診断：

```python
import os
print(f"現在のディレクトリ: {os.getcwd()}")
print(f"myprojectが存在: {os.path.exists('myproject')}")
if not os.path.exists('myproject'):
    print("リポジトリクローンに失敗している可能性があります。セクション1を再実行してください。")
```

**2. `requirements.txt` インストールエラー**
- Colab環境で必要なパッケージが既にインストール済みの場合があります
- エラーが出た場合は個別にインストール: `!pip install numpy scipy matplotlib pandas tqdm`

**3. 並列実験エラー**
- 並列実験はオプションです。基本実験（セクション4-5）は単体で動作します


In [ ]:
# Calvano et al. (2020) Q-Learning Replication (Colab)

このノートブックは Calvano et al. (2020) の Q学習による価格付け実験を Google Colab 上で再現します。

**論文**: Calvano, E., Calzolari, G., Denicolò, V., & Pastorello, S. (2020). Artificial Intelligence, Algorithmic Pricing, and Collusion. *American Economic Review*, 110(10), 3267-3297.

---


In [ ]:
## 1. セットアップ（リポジトリのクローン＆依存パッケージのインストール）


In [ ]:
# リポジトリをクローン
!git clone https://github.com/Yusei406/calvano-thesis.git
%cd calvano-thesis

# 依存パッケージをインストール
!pip install -r requirements.txt


In [ ]:
## 2. モジュールのインポート


In [ ]:
import sys
sys.path.append('.')

from myproject.env import DemandEnvironment
from myproject.agent import QLearningAgent
from myproject.train import train_agents
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

print("✅ モジュールのインポート完了")


In [ ]:
## 3. 環境セットアップとベンチマーク計算（Nash/協調均衡）


In [ ]:
# Calvanoパラメータで環境を作成
env = DemandEnvironment(
    demand_intercept=0.0,   # a_0 (outside option)
    product_quality=2.0,    # a_i (product quality)
    demand_slope=0.25,      # μ (price sensitivity)
    marginal_cost=1.0       # c (marginal cost)
)

# Nash均衡・協調均衡を計算
nash_eq = env.get_nash_equilibrium()
coop_eq = env.get_collusive_outcome()

print('📊 ベンチマーク均衡の計算結果:')
print('Nash Equilibrium:')
print(f'  Price: {nash_eq["prices"][0]:.3f}')
print(f'  Individual Profit: {nash_eq["individual_profit"]:.3f}')
print(f'  Joint Profit: {nash_eq["joint_profit"]:.3f}')
print()
print('Cooperative Equilibrium:')
print(f'  Price: {coop_eq["prices"][0]:.3f}')
print(f'  Individual Profit: {coop_eq["individual_profit"]:.3f}')
print(f'  Joint Profit: {coop_eq["joint_profit"]:.3f}')


In [ ]:
## 4. Q学習エージェントの学習実験

⚠️ **重要**: フル実験は8-12時間かかります。まずはクイックテストを実行してください。


In [ ]:
# 1. クイックテスト（約5分）
print('🚀 クイックテスト実験を開始...')
agents, env, history = train_agents(
    n_episodes=100,
    iterations_per_episode=1000,
    learning_rate=0.15,
    discount_factor=0.95,
    epsilon_decay_beta=4e-6,
    memory_length=1,
    verbose=True
)

# 2. フル実験（8-12時間）- コメントアウトを外して使用
# print('🚀 フル実験（論文レベル）を開始...')
# print('⚠️  予想実行時間: 8-12時間')
# agents, env, history = train_agents(
#     n_episodes=50000,
#     iterations_per_episode=25000,
#     learning_rate=0.15,
#     discount_factor=0.95,
#     epsilon_decay_beta=4e-6,
#     memory_length=1,
#     verbose=True
# )

print('\\n✅ 学習完了!')
print(f'Final individual profit: {history["training_summary"]["final_individual_profit"]:.4f}')
print(f'Nash ratio: {history["training_summary"]["nash_ratio_individual"]:.3f}')


In [ ]:
## 5. 学習履歴の可視化


In [ ]:
# 学習結果のプロット
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 個別利益の推移
axes[0, 0].plot(history['episodes'], history['individual_profits'], 'b-', linewidth=2)
axes[0, 0].axhline(y=nash_eq['individual_profit'], color='r', linestyle='--', label='Nash', linewidth=2)
axes[0, 0].axhline(y=coop_eq['individual_profit'], color='g', linestyle='--', label='Cooperative', linewidth=2)
axes[0, 0].set_title('Individual Profits', fontsize=14)
axes[0, 0].set_xlabel('Episode')
axes[0, 0].set_ylabel('Profit')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 合計利益の推移
axes[0, 1].plot(history['episodes'], history['joint_profits'], 'b-', linewidth=2)
axes[0, 1].axhline(y=nash_eq['joint_profit'], color='r', linestyle='--', label='Nash', linewidth=2)
axes[0, 1].axhline(y=coop_eq['joint_profit'], color='g', linestyle='--', label='Cooperative', linewidth=2)
axes[0, 1].set_title('Joint Profits', fontsize=14)
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Nash比
nash_ratios = [p / nash_eq['individual_profit'] for p in history['individual_profits']]
axes[1, 0].plot(history['episodes'], nash_ratios, 'orange', linewidth=2)
axes[1, 0].axhline(y=1.0, color='r', linestyle='--', label='Nash', linewidth=2)
axes[1, 0].set_title('Individual Profit / Nash Profit', fontsize=14)
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Epsilon減衰
axes[1, 1].plot(history['episodes'], history['epsilon_values'], 'purple', linewidth=2)
axes[1, 1].set_title('Epsilon Decay', fontsize=14)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print('📈 最終結果:')
print(f'Individual Profit: {history["training_summary"]["final_individual_profit"]:.4f}')
print(f'Nash Ratio: {history["training_summary"]["nash_ratio_individual"]:.3f}')


In [ ]:
## 6. 並列実験でTable A.2を再現

**論文の完全再現**を行う場合は、以下のフル実験を実行してください：


In [ ]:
# 1. クイックテスト並列実験（約10分）
print("🚀 クイックテスト並列実験を開始...")
!python -m myproject.scripts.table_a2_parallel --episodes 1000 --n-seeds 2 --n-sessions 2 --max-workers 2

# 2. フル実験（8-12時間）- 論文の完全再現（コメントアウトを外して使用）
# print("🚀 フル並列実験（Table A.2完全再現）を開始...")
# print("📊 パラメータ: 50,000エピソード × 10シード")
# print("⏰ 予想実行時間: 8-12時間")
# !python -m myproject.scripts.table_a2_parallel --episodes 50000 --n-seeds 10 --n-sessions 4 --max-workers 4

print("✅ 並列実験完了!")

# 結果の確認とダウンロード
import os
from google.colab import files

if os.path.exists('results'):
    for file in os.listdir('results'):
        if file.endswith('.json'):
            print(f"📊 結果ファイル: {file}")
            files.download(f'results/{file}')
            break


In [ ]:
# Calvano et al. (2020) Q-Learning Replication (Colab)

このノートブックは Calvano et al. (2020) の Q学習による価格付け実験を Google Colab 上で再現します。

**論文**: Calvano, E., Calzolari, G., Denicolò, V., & Pastorello, S. (2020). Artificial Intelligence, Algorithmic Pricing, and Collusion. *American Economic Review*, 110(10), 3267-3297.

---


In [ ]:
## 1. セットアップ（リポジトリのクローン＆依存パッケージのインストール）


In [ ]:
# リポジトリをクローン
!git clone https://github.com/Yusei406/calvano-thesis.git
%cd calvano-thesis

# 依存パッケージをインストール
!pip install -r requirements.txt


In [ ]:
## 2. モジュールのインポート


In [ ]:
import sys
sys.path.append('.')

from myproject.env import DemandEnvironment
from myproject.agent import QLearningAgent
from myproject.train import train_agents
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

print("✅ モジュールのインポート完了")


In [ ]:
## 3. 環境セットアップとベンチマーク計算（Nash/協調均衡）


In [ ]:
# Calvanoパラメータで環境を作成
env = DemandEnvironment(
    demand_intercept=0.0,   # a_0 (outside option)
    product_quality=2.0,    # a_i (product quality)
    demand_slope=0.25,      # μ (price sensitivity)
    marginal_cost=1.0       # c (marginal cost)
)

# Nash均衡・協調均衡を計算
nash_eq = env.get_nash_equilibrium()
coop_eq = env.get_collusive_outcome()

print('📊 ベンチマーク均衡の計算結果:')
print()
print('Nash Equilibrium:')
print(f'  Price: {nash_eq["prices"][0]:.3f}')
print(f'  Individual Profit: {nash_eq["individual_profit"]:.3f}')
print(f'  Joint Profit: {nash_eq["joint_profit"]:.3f}')
print()
print('Cooperative Equilibrium:')
print(f'  Price: {coop_eq["prices"][0]:.3f}')
print(f'  Individual Profit: {coop_eq["individual_profit"]:.3f}')
print(f'  Joint Profit: {coop_eq["joint_profit"]:.3f}')


In [ ]:
## 4. Q学習エージェントの学習実験

⚠️ **重要**: 下記のコメントアウトを調整して実行レベルを選択してください


In [ ]:
# 1. クイックテスト（約5分）
print('🚀 クイックテスト実験を開始...')
agents, env, history = train_agents(
    n_episodes=100,
    iterations_per_episode=1000,
    learning_rate=0.15,
    discount_factor=0.95,
    epsilon_decay_beta=4e-6,
    memory_length=1,
    verbose=True
)

# 2. 中規模実験（約30分）- コメントアウトを外して使用
# print('🚀 中規模実験を開始...')
# agents, env, history = train_agents(
#     n_episodes=1000,
#     iterations_per_episode=5000,
#     learning_rate=0.15,
#     discount_factor=0.95,
#     epsilon_decay_beta=4e-6,
#     memory_length=1,
#     verbose=True
# )

# 3. フル実験（8-12時間）- 論文の完全再現（コメントアウトを外して使用）
# print('🚀 フル実験（論文レベル）を開始...')
# print('⚠️  予想実行時間: 8-12時間')
# agents, env, history = train_agents(
#     n_episodes=50000,
#     iterations_per_episode=25000,
#     learning_rate=0.15,
#     discount_factor=0.95,
#     epsilon_decay_beta=4e-6,
#     memory_length=1,
#     verbose=True
# )

print('\n✅ 学習完了!')
print(f'Final individual profit: {history["training_summary"]["final_individual_profit"]:.4f}')
print(f'Final joint profit: {history["training_summary"]["final_joint_profit"]:.4f}')
print(f'Nash ratio (individual): {history["training_summary"]["nash_ratio_individual"]:.3f}')


In [ ]:
## 5. 学習履歴の可視化


In [ ]:
# 学習結果のプロット
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 個別利益の推移
axes[0, 0].plot(history['episodes'], history['individual_profits'], 'b-', linewidth=2)
axes[0, 0].axhline(y=nash_eq['individual_profit'], color='r', linestyle='--', label='Nash', linewidth=2)
axes[0, 0].axhline(y=coop_eq['individual_profit'], color='g', linestyle='--', label='Cooperative', linewidth=2)
axes[0, 0].set_title('Individual Profits', fontsize=14)
axes[0, 0].set_xlabel('Episode')
axes[0, 0].set_ylabel('Profit')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 合計利益の推移
axes[0, 1].plot(history['episodes'], history['joint_profits'], 'b-', linewidth=2)
axes[0, 1].axhline(y=nash_eq['joint_profit'], color='r', linestyle='--', label='Nash', linewidth=2)
axes[0, 1].axhline(y=coop_eq['joint_profit'], color='g', linestyle='--', label='Cooperative', linewidth=2)
axes[0, 1].set_title('Joint Profits', fontsize=14)
axes[0, 1].set_xlabel('Episode')
axes[0, 1].set_ylabel('Profit')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Epsilon減衰
axes[1, 0].plot(history['episodes'], history['epsilon_values'], 'purple', linewidth=2)
axes[1, 0].set_title('Epsilon Decay', fontsize=14)
axes[1, 0].set_xlabel('Episode')
axes[1, 0].set_ylabel('Epsilon')
axes[1, 0].grid(True, alpha=0.3)

# Nash比
nash_ratios = [p / nash_eq['individual_profit'] for p in history['individual_profits']]
axes[1, 1].plot(history['episodes'], nash_ratios, 'orange', linewidth=2)
axes[1, 1].axhline(y=1.0, color='r', linestyle='--', label='Nash', linewidth=2)
axes[1, 1].axhline(y=coop_eq['individual_profit'] / nash_eq['individual_profit'], color='g', linestyle='--', label='Cooperative', linewidth=2)
axes[1, 1].set_title('Individual Profit / Nash Profit', fontsize=14)
axes[1, 1].set_xlabel('Episode')
axes[1, 1].set_ylabel('Ratio')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 最終結果の表示
print('📈 最終結果:')
print(f'Individual Profit: {history["training_summary"]["final_individual_profit"]:.4f} (Nash比: {history["training_summary"]["nash_ratio_individual"]:.3f})')
print(f'Joint Profit: {history["training_summary"]["final_joint_profit"]:.4f}')
print(f'Final Epsilon: {history["training_summary"]["final_epsilon"]:.4f}')


In [ ]:
## 6. （オプション）並列実験でTable A.2を再現

大規模実験を行いたい場合は、以下のコマンドを実行してください：


In [ ]:
# 1. クイックテスト並列実験（約10分）
print("🚀 クイックテスト並列実験を開始...")
!python -m myproject.scripts.table_a2_parallel --episodes 1000 --n-seeds 2 --n-sessions 2 --max-workers 2

# 2. 中規模実験（約60分）- コメントアウトを外して実行
# print("🚀 中規模並列実験を開始...")
# !python -m myproject.scripts.table_a2_parallel --episodes 5000 --n-seeds 5 --n-sessions 2 --max-workers 4

# 3. フル実験（8-12時間）- 論文の完全再現（コメントアウトを外して実行）
# print("🚀 フル並列実験（Table A.2完全再現）を開始...")
# print("📊 パラメータ: 50,000エピソード × 10シード")
# print("⏰ 予想実行時間: 8-12時間")
# !python -m myproject.scripts.table_a2_parallel --episodes 50000 --n-seeds 10 --n-sessions 4 --max-workers 4 --output-dir results

print("✅ 並列実験完了!")


In [ ]:
## 7. 結果のダウンロードと分析

並列実験を実行した場合、結果をダウンロードして分析できます：


In [ ]:
# 結果ファイルの確認とダウンロード
import os
import json
from google.colab import files

# 結果ディレクトリの確認
print("📁 実験結果ファイル:")
if os.path.exists('results'):
    result_files = []
    for file in os.listdir('results'):
        if file.endswith('.json'):
            print(f"  📊 {file}")
            result_files.append(file)
        elif file.endswith('.csv'):
            print(f"  📈 {file}")
    
    # 最新の結果を分析
    if result_files:
        latest_result = max(result_files, key=lambda f: os.path.getctime(f'results/{f}'))
        print(f"\n🔍 最新結果の分析: {latest_result}")
        
        with open(f'results/{latest_result}', 'r') as f:
            data = json.load(f)
        
        # Table A.2形式の結果表示
        if 'sessions' in data:
            df = pd.DataFrame(data['sessions'])
            if not df.empty:
                print(f"\n📊 Table A.2 スタイルの結果:")
                print(f"シード数: {len(df['seed'].unique())}")
                print(f"セッション数: {len(df)}")
                print(f"個別利益: {df['final_individual_profit'].mean():.4f} ± {df['final_individual_profit'].std():.4f}")
                print(f"合計利益: {df['final_joint_profit'].mean():.4f} ± {df['final_joint_profit'].std():.4f}")
                print(f"Nash比 (個別): {df['nash_ratio_individual'].mean():.3f}")
        
        # ダウンロード実行
        print(f"\n💾 結果ファイルをダウンロード中...")
        files.download(f'results/{latest_result}')
        
else:
    print("  ❌ 結果ファイルがありません。まず並列実験を実行してください。")

print("\n✅ ダウンロード完了!")


In [ ]:
## 8. 📋 論文レベル実験の実行手順

**Calvano et al. (2020) Table A.2の完全再現**を行う場合の推奨手順：

### ステップ1: 動作確認
- まず**クイックテスト**（セクション4 & 6）を実行して動作を確認

### ステップ2: 中規模実験
- **中規模実験**（セクション6）のコメントアウトを外して実行（約60分）
- 結果をダウンロードして確認

### ステップ3: フル実験
- **フル実験**（セクション6）のコメントアウトを外して実行
- ⚠️ **8-12時間**かかるため、時間に余裕があるときに実行
- 実行パラメータ：
  - **50,000エピソード × 10シード**
  - **25,000 iterations per episode**
  - **論文と同一のパラメータ**

### 期待される結果（論文Table A.2）
- **個別利益**: 0.18 ± 0.03
- **合計利益**: 0.26 ± 0.04  
- **Nash比**: 約1.2-1.4 (協調的行動の証拠)

### 💡 ヒント
- Colab Pro/Pro+の使用を推奨（タイムアウト防止）
- 実験中はタブを閉じないでください
- 結果は自動的に`results/`に保存されます
